# 📊 Notebook 04: Data Transformation

**Autor:** Gian  
**Fecha:** 2026-01-19  
**Objetivo:** Transformar el dataset limpio para prepararlo para el modelado  

---

## 📋 Contenido

1. Configuración del entorno
2. Carga de datos limpios
3. Encoding de variables categóricas
4. Escalado de variables numéricas
5. Transformaciones matemáticas
6. Generación de dataset transformado
7. Reporte de transformación

---
## 1. Configuración del Entorno

In [37]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import warnings

# Configuración
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 8)

# Seed
np.random.seed(42)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


---
## 2. Carga de Datos Limpios

In [38]:
# Rutas
OUTPUT_PATH = Path("../../outputs/gian")
CLEAN_DATA_PATH = OUTPUT_PATH / "data" / "data_clean.csv"
TRANSFORMED_DATA_PATH = OUTPUT_PATH / "data"

# Cargar datos limpios
df = pd.read_csv(CLEAN_DATA_PATH)

print(f"✅ Dataset cargado: {df.shape[0]:,} registros × {df.shape[1]} columnas")
print(f"💾 Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

✅ Dataset cargado: 9,701 registros × 67 columnas
💾 Memoria: 18.71 MB


In [39]:
# Crear copia para transformación
df_transformed = df.copy()

print(f"✅ Copia creada para transformación")
print(f"📊 Shape: {df_transformed.shape}")

✅ Copia creada para transformación
📊 Shape: (9701, 67)


---
## 3. Encoding de Variables Categóricas

### 3.1. Identificar Variables Categóricas

In [40]:
# Identificar columnas categóricas
categorical_cols = df_transformed.select_dtypes(include=["object"]).columns.tolist()

# Excluir columnas que no necesitan encoding
exclude_cols = [
    "cliente_id",  # ID único
    "pais", "ciudad", "borough", "estado",  # Ubicaciones (no usar para modelado)
    "fecha_registro", "fecha_ultimo_pago", "ultimo_contacto_soporte"  # Fechas (se procesarán después)
]
categorical_cols = [col for col in categorical_cols if col not in exclude_cols]

print(f"📊 Variables categóricas a encodear: {len(categorical_cols)}")
print()
for i, col in enumerate(categorical_cols, 1):
    unique_vals = df_transformed[col].nunique()
    print(f"  {i:2d}. {col:30s} - {unique_vals} categorías")

📊 Variables categóricas a encodear: 23

   1. genero                         - 2 categorías
   2. segmento_cliente               - 3 categorías
   3. tiene_pareja                   - 2 categorías
   4. tiene_dependientes             - 2 categorías
   5. tipo_contrato                  - 3 categorías
   6. metodo_pago                    - 4 categorías
   7. canal_registro                 - 1 categorías
   8. descuento_aplicado             - 5 categorías
   9. aumento_precio_3m              - 2 categorías
  10. facturacion_sin_papel          - 2 categorías
  11. servicio_telefono              - 2 categorías
  12. lineas_multiples               - 3 categorías
  13. tipo_internet                  - 3 categorías
  14. seguridad_online               - 3 categorías
  15. respaldo_online                - 3 categorías
  16. proteccion_dispositivo         - 3 categorías
  17. soporte_tecnico                - 3 categorías
  18. streaming_tv                   - 3 categorías
  19. streaming_pelicula

### 3.2. Label Encoding para Variables Binarias

In [41]:
# Variables binarias (Si/No, Masculino/Femenino, etc.)
binary_cols = []
for col in categorical_cols:
    if df_transformed[col].nunique() == 2:
        binary_cols.append(col)

print(f"🔧 Aplicando Label Encoding a {len(binary_cols)} variables binarias...")
print()

label_encoders = {}
for col in binary_cols:
    le = LabelEncoder()
    df_transformed[col + "_encoded"] = le.fit_transform(df_transformed[col].astype(str))
    label_encoders[col] = le
    print(f"  ✅ {col}: {list(le.classes_)} → {list(range(len(le.classes_)))}")

print()
print(f"✅ {len(binary_cols)} variables binarias encodeadas")

🔧 Aplicando Label Encoding a 6 variables binarias...

  ✅ genero: ['Femenino', 'Masculino'] → [0, 1]
  ✅ tiene_pareja: ['No', 'Si'] → [0, 1]
  ✅ tiene_dependientes: ['No', 'Si'] → [0, 1]
  ✅ aumento_precio_3m: ['No', 'Si'] → [0, 1]
  ✅ facturacion_sin_papel: ['No', 'Si'] → [0, 1]
  ✅ servicio_telefono: ['No', 'Si'] → [0, 1]

✅ 6 variables binarias encodeadas


# Variables con más de 2 categorías
multi_categorical_cols = [col for col in categorical_cols if col not in binary_cols]

# ⚠️ EXCLUIR columnas de fechas (ya están en formato datetime)
date_cols = ["fecha_registro", "fecha_ultimo_pago", "ultimo_contacto_soporte"]
multi_categorical_cols = [col for col in multi_categorical_cols if col not in date_cols]

print(f"🔧 Aplicando One-Hot Encoding a {len(multi_categorical_cols)} variables...")
print()
print("Variables a encodear:")
for col in multi_categorical_cols:
    print(f"  - {col}: {df_transformed[col].nunique()} categorías")
print()


In [42]:
# Variables con más de 2 categorías
multi_categorical_cols = [col for col in categorical_cols if col not in binary_cols]

print(f"🔧 Aplicando One-Hot Encoding a {len(multi_categorical_cols)} variables...")
print()

# Aplicar One-Hot Encoding
df_transformed = pd.get_dummies(df_transformed, 
                                 columns=multi_categorical_cols,
                                 prefix=multi_categorical_cols,
                                 drop_first=True)  # Evitar multicolinealidad

print(f"✅ One-Hot Encoding aplicado")
print(f"📊 Nuevas columnas creadas: {df_transformed.shape[1] - df.shape[1]}")
print(f"📊 Shape actual: {df_transformed.shape}")

🔧 Aplicando One-Hot Encoding a 17 variables...

✅ One-Hot Encoding aplicado
📊 Nuevas columnas creadas: 34
📊 Shape actual: (9701, 101)


---
## 4. Escalado de Variables Numéricas

### 4.1. Identificar Variables Numéricas

In [43]:
# Identificar columnas numéricas
numeric_cols = df_transformed.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Excluir columnas que no necesitan escalado
exclude_numeric = ["cliente_id", "cancelacion", "es_mayor", "codigo_postal",
                   "latitud", "longitud"]  # IDs, target, flags, coordenadas

# También excluir las columnas encoded que acabamos de crear
exclude_numeric.extend([col + "_encoded" for col in binary_cols])

numeric_cols = [col for col in numeric_cols if col not in exclude_numeric]

print(f"📊 Variables numéricas a escalar: {len(numeric_cols)}")
print()
for i, col in enumerate(numeric_cols[:10], 1):  # Mostrar solo las primeras 10
    print(f"  {i:2d}. {col}")
if len(numeric_cols) > 10:
    print(f"  ... y {len(numeric_cols) - 10} más")

📊 Variables numéricas a escalar: 31

   1. edad
   2. ingreso_mediano
   3. densidad_poblacional
   4. antiguedad
   5. cargo_mensual
   6. ingresos_totales
   7. errores_pago
   8. score_riesgo
   9. conexiones_mensuales
  10. dias_activos_semanales
  ... y 21 más


### 4.2. Aplicar StandardScaler (Z-score normalization)

In [44]:
# Aplicar StandardScaler
print("🔧 Aplicando StandardScaler (media=0, std=1)...")
print()

scaler = StandardScaler()
df_transformed[numeric_cols] = scaler.fit_transform(df_transformed[numeric_cols])

print("✅ StandardScaler aplicado")
print()
print("📊 Verificación (primeras 5 columnas):")
for col in numeric_cols[:5]:
    mean = df_transformed[col].mean()
    std = df_transformed[col].std()
    print(f"  {col:30s} - Media: {mean:7.4f}, Std: {std:7.4f}")

🔧 Aplicando StandardScaler (media=0, std=1)...

✅ StandardScaler aplicado

📊 Verificación (primeras 5 columnas):
  edad                           - Media: -0.0000, Std:  1.0001
  ingreso_mediano                - Media:  0.0000, Std:  1.0001
  densidad_poblacional           - Media: -0.0000, Std:  1.0001
  antiguedad                     - Media:  0.0000, Std:  1.0001
  cargo_mensual                  - Media:  0.0000, Std:  1.0001


---
## 5. Transformaciones Matemáticas

### 5.1. Identificar Variables Sesgadas

In [45]:
# Calcular skewness de variables numéricas ANTES de escalar
# (usamos el dataset original para esto)
print("📊 Analizando sesgo (skewness) de variables numéricas...")
print()

skewness = df[numeric_cols].skew().sort_values(ascending=False)
highly_skewed = skewness[abs(skewness) > 1].index.tolist()

print(f"Variables con alto sesgo (|skew| > 1): {len(highly_skewed)}")
print()
for col in highly_skewed[:10]:  # Mostrar las 10 más sesgadas
    print(f"  {col:30s} - Skewness: {skewness[col]:7.2f}")

print()
print("💡 Nota: Estas variables podrían beneficiarse de transformación log")

📊 Analizando sesgo (skewness) de variables numéricas...

Variables con alto sesgo (|skew| > 1): 10

  errores_pago                   - Skewness:    4.22
  escaladas                      - Skewness:    3.91
  tickets_soporte                - Skewness:    2.96
  dias_ultima_conexion           - Skewness:    2.48
  referencias_hechas             - Skewness:    2.06
  score_riesgo                   - Skewness:    1.63
  downgrade_reciente             - Skewness:    1.40
  ingresos_totales               - Skewness:    1.36
  dias_mora                      - Skewness:    1.23
  cambio_plan_reciente           - Skewness:    1.12

💡 Nota: Estas variables podrían beneficiarse de transformación log


### 5.2. Aplicar Transformación Log (Opcional)

In [46]:
# NOTA: Esta sección es opcional y se puede aplicar en Feature Engineering
# Por ahora, solo documentamos las variables que podrían beneficiarse

print("📝 Variables candidatas para transformación log:")
print()
for col in highly_skewed[:5]:
    print(f"  - {col}")
print()
print("💡 Estas transformaciones se aplicarán en el Notebook 06 (Feature Engineering)")

📝 Variables candidatas para transformación log:

  - errores_pago
  - escaladas
  - tickets_soporte
  - dias_ultima_conexion
  - referencias_hechas

💡 Estas transformaciones se aplicarán en el Notebook 06 (Feature Engineering)


---
## 6. Generación de Dataset Transformado

In [47]:
# Guardar dataset transformado
transformed_file = TRANSFORMED_DATA_PATH / "04_data_transformed.csv"

df_transformed.to_csv(transformed_file, index=False)

print(f"✅ Dataset transformado guardado en: {transformed_file}")
print()
print(f"📊 Shape final: {df_transformed.shape}")
print(f"💾 Tamaño: {transformed_file.stat().st_size / 1024**2:.2f} MB")

✅ Dataset transformado guardado en: ../../outputs/gian/data/04_data_transformed.csv

📊 Shape final: (9701, 101)
💾 Tamaño: 9.45 MB


---
## 7. Reporte de Transformación

In [48]:
# Crear reporte de transformación
report = {
    "Registros_Inicial": len(df),
    "Registros_Final": len(df_transformed),
    "Columnas_Inicial": df.shape[1],
    "Columnas_Final": df_transformed.shape[1],
    "Columnas_Agregadas": df_transformed.shape[1] - df.shape[1],
    "Variables_Binarias_Encoded": len(binary_cols),
    "Variables_OneHot_Encoded": len(multi_categorical_cols),
    "Variables_Escaladas": len(numeric_cols),
    "Variables_Sesgadas_Detectadas": len(highly_skewed),
}

report_df = pd.DataFrame(report, index=[0]).T
report_df.columns = ["Valor"]

# Guardar reporte
report_file = OUTPUT_PATH / "reports" / "04_transformation_report.csv"
report_file.parent.mkdir(parents=True, exist_ok=True)
report_df.to_csv(report_file)

print("📋 Reporte de Transformación:")
print()
print(report_df)
print()
print(f"✅ Reporte guardado en: {report_file}")

📋 Reporte de Transformación:

                               Valor
Registros_Inicial               9701
Registros_Final                 9701
Columnas_Inicial                  67
Columnas_Final                   101
Columnas_Agregadas                34
Variables_Binarias_Encoded         6
Variables_OneHot_Encoded          17
Variables_Escaladas               31
Variables_Sesgadas_Detectadas     10

✅ Reporte guardado en: ../../outputs/gian/reports/04_transformation_report.csv


---
## 8. Resumen de Transformaciones

### ✅ Transformaciones Aplicadas

1. **Encoding de Variables Categóricas:**
   - Label Encoding para variables binarias (Si/No, etc.)
   - One-Hot Encoding para variables con múltiples categorías

2. **Escalado de Variables Numéricas:**
   - StandardScaler aplicado (media=0, std=1)
   - Variables normalizadas para el modelado

3. **Análisis de Sesgo:**
   - Identificadas variables con alto sesgo
   - Candidatas para transformación log en Feature Engineering

### 📊 Dataset Transformado

- **Ubicación:** 
- **Registros:** Sin pérdida
- **Columnas:** Aumentadas por One-Hot Encoding
- **Estado:** Listo para EDA y Feature Engineering

### 🎯 Próximo Paso

**Notebook 05: EDA (Exploratory Data Analysis)**
- Análisis exploratorio profundo
- Visualizaciones de distribuciones
- Análisis de correlaciones
- Identificación de patrones